In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from pytz import timezone

def get_airports():
    
    schema = "gans_local"
    host = "localhost"
    user = "root"
    password = mySQL_pass
    port = 3306
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    cities_df = pd.read_sql("cities", con=connection_string)
    
    list_for_df = []

    for _, city in cities_df.iterrows():
        latitude = city["Latitude"]
        longitude = city["Longitude"]
        city_id = city["City_id"]

        url = "https://aerodatabox.p.rapidapi.com/airports/search/location"

        querystring = {"lat":latitude,"lon":longitude,"radiusKm":"50","limit":"5","withFlightInfoOnly":"true"}
        headers = {
          "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
          "X-RapidAPI-Key": api_key #USE THE PROVIDED KEY
        }

        response = requests.get(url, headers=headers, params=querystring)

        for item in response.json()['items']:
            airports_data = {
            "city_id": city_id,
            "icao": item.get("icao", None),
            "municipality_name": item.get("municipalityName", None)
        }


        list_for_df.append(airports_data)

    airports_df = pd.DataFrame(list_for_df)

    return airports_df  

In [ ]:
airports_df = get_airports()
airports_df

In [ ]:
airports_df.to_csv('airports_to_db.csv', index=False)

In [ ]:
airports_df.to_sql('airports',
                  if_exists='append',
                  con=connection_string,
                  index=False)